In [1]:
import numpy as np
import sympy as sp
import pandas as pd
import optuna
import matplotlib.pyplot as plt

from EKF import ExtendedKalmanFilter as EKF

c:\Users\alefi\anaconda3\envs\CasADi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Symbols definition

In [2]:
#States
Tz, Tw, Tt, beta, Uwall = sp.symbols('Tz Tw Tt beta Uwall')

#Inputs
Cf, Tz_ref, Te = sp.symbols('Cf Tz_ref Te')

#Parameters
Cz, Uwin, alpha, Cwall, mw, cw, Pmax, sigma, COP, T_shift, Np = sp.symbols('Cz Uwin alpha Cwall mw cw Pmax sigma COP T_shift Np')

# Equations definition

In [3]:
# Define the state equations, output equations, state symbols, input symbols, and parameters

beta_nl = beta/(1+sp.exp(-sigma*(Tz_ref-Tz-T_shift)))

state_equations = [Tz +1/Cz*(Uwall*(Tw-Tz)+Uwin*(Te-Tz)+alpha*Np+beta_nl*(Tt-Tz)),
                    Tw+1/Cwall*(Uwall*(Te-Tw)+Uwall*(Tz-Tw)),
                    Tt+1/(mw*cw)*(Pmax*COP*Cf + beta_nl*(Tz-Tt)),
                    beta,
                    Uwall
                    ]

output_equation = [Tz,
                   Tt
                   ]

state_symbols = [Tz, Tw, Tt, beta, Uwall]
input_symbols = [Cf, Tz_ref, Te]

params_dict = { "Cz" : 6.2608e8,
                "Uwin" : 1.1362e5,
                "alpha" : 3.4024e3,
                "Cwall" : 8.1597e9,
                "mw" : 9.788e3,
                "cw" : 4186,
                "Pmax" : 8.58e6,
                "sigma" : 2.2,
                "COP" : 5.4,
                "T_shift" : 0,
                "Np" : 0
                }  

P_0 = np.diag([0.1, 0.1, 0.1, 1e5, 1e5])  # Initial state covariance estimate

# Create an instance of the ExtendedKalmanFilter class
ekf = EKF(state_equations, output_equation, state_symbols, input_symbols, params_dict, P_0)

# Reading data

In [4]:
df = pd.read_csv('Data\Data_expl_nom.csv')

Cf = df['Cf_0']
Tz_ref_anti = df['Tz_ref_anti']
Te = df['Te']
Np = df['Np']
Tz_mean = df['Tz_mean']
Tt_mean = df['Tt_mean']
Twall_mean = df['Twall_mean']
beta_true = df['beta'][0]
Uwall_true = df['Uwall'][0]

# Optuna optimization


In [5]:
def objective(trial):
    # Define the search spaces for the three parameters
    q1 = trial.suggest_float('q1', 1e-2, 1e3, log=True)
    q2 = trial.suggest_float('q2', 1e-2, 1e2, log=True)
    q3 = trial.suggest_float('q3', 1e-2, 1e3, log=True)
    q4 = trial.suggest_float('q4', 1e8, 1e9, log=True)
    q5 = trial.suggest_float('q5', 1e9, 1e12, log=True)

    # Create the Q_k matrix with the suggested parameters
    Q_k = np.diag([q1, q2, q3, q4, q5])
    R_k = np.diag([0.1, 0.1])

    ekf.set_QR(Q_k, R_k)

    # Initial guesses
    beta_init = 950000
    Uwall_init = 5.5955e5

    # Initial conditions
    y_k = np.array([Tz_mean, Tt_mean]).T
    u_k = np.array([Cf, Tz_ref_anti, Te]).T
    x_k_minus_1 = np.array([Tz_mean[0], Twall_mean[0], Tt_mean[0], beta_init, Uwall_init])

    # Initializing errors
    beta_err = 0.0
    Uwall_err = 0.0

    time_steps = 500

    # Run the Extended Kalman Filter
    x_k_tot = np.empty((time_steps, len(state_symbols)))

    for k in range(time_steps):

        x_k = ekf.ekf(x_k_minus_1, [u_k[k]], y_k[k])

        x_k_tot[k] = x_k
        
        x_k_minus_1 = x_k

        beta_err += (float(x_k[3]) - beta_true)**2
        Uwall_err += (float(x_k[4]) - Uwall_true)**2
        
    mse = np.linalg.norm(np.array([np.sqrt(beta_err), np.sqrt(Uwall_err)]))

    return mse

# Optuna main

In [6]:
# Create a study object
study = optuna.create_study(direction='minimize')

# Optimize the objective function
study.optimize(objective, n_trials=10)

# Print the best trial and its parameters
print('Best trial:')
trial = study.best_trial
print(f'Value: {trial.value}')
print(f'Params: {trial.params}')

[I 2024-05-17 15:20:02,052] A new study created in memory with name: no-name-f2e6d43c-9236-4046-aeb9-bc6a24806a38
[I 2024-05-17 15:20:53,548] Trial 0 finished with value: 9745431.447072778 and parameters: {'q1': 7.015625144477222, 'q2': 67.4376374990372, 'q3': 0.7367247622866347, 'q4': 108337983.85224748, 'q5': 1395367824.999297}. Best is trial 0 with value: 9745431.447072778.
[I 2024-05-17 15:21:47,849] Trial 1 finished with value: 9959676.942166546 and parameters: {'q1': 0.13750849907095278, 'q2': 0.19610444748590922, 'q3': 1.8152844558285397, 'q4': 678427690.6235548, 'q5': 616037455048.0607}. Best is trial 0 with value: 9745431.447072778.
[I 2024-05-17 15:22:43,081] Trial 2 finished with value: 7388672.133869476 and parameters: {'q1': 0.010470127959734342, 'q2': 0.9391975061601794, 'q3': 314.18458153758917, 'q4': 276443579.9087478, 'q5': 757193805382.4327}. Best is trial 2 with value: 7388672.133869476.
[I 2024-05-17 15:23:44,621] Trial 3 finished with value: 18870220.98924689 and p

Best trial:
Value: 7388672.133869476
Params: {'q1': 0.010470127959734342, 'q2': 0.9391975061601794, 'q3': 314.18458153758917, 'q4': 276443579.9087478, 'q5': 757193805382.4327}
